In [1]:
import nltk; nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import string
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
import pyLDAvis
import pyLDAvis.gensim_models  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

# Enable logging for gensim - optional
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.ERROR)

import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/cbeams/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
# NLTK Stop words
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
stop_words.extend(['from', 'subject', 're', 'edu', 'use'])

In [3]:
# Import Dataset
df = pd.read_json('../raw_data/all_data_df.json')
print(df.artist.unique())
df.head()

['Adam Devine' 'Adam Hess' 'Adam Hills' 'Adam Sandler' 'Adel Karam'
 'Aisling Bea' 'Al Madrigal' 'Al Murray' 'Al Porter' 'Alan Carr'
 'Alex Edelman' 'Ali Wong' 'Alistair McGowan' 'Amanda Seales'
 'Amy Schumer' 'Andi Osho' 'Andrew Lawrence' 'Andrew Maxwell'
 'Andy Parsons' 'Andy Woodhull' 'Angela Barnes' 'Anjelah Johnson'
 'Anthony Jeselnik' 'Ari Shaffir' 'Arj Barker' 'Arsenio Hall'
 'Aziz Ansari' 'Bert Kreischer' 'Big Jay Oakerson' 'Bill Burr'
 'Bill Hicks' 'Bill Maher' 'Billy Connolly' 'Bo Burnham' 'Bob Monkhouse'
 'Brad Williams' 'Brent Morin' 'Brian Regan' 'Bridget Everett'
 'Cedric The Entertainer' 'Celia Pacquola' 'Chelsea Peretti'
 'Chris Addison' "Chris D'Elia" 'Chris Gethard' 'Chris McCausland'
 'Chris Moyles' 'Chris Ramsey' 'Chris Rock' 'Chris Tucker'
 'Christina Pazsitzky' 'Colin Quinn' 'Craig Ferguson' 'Cristela Alonzo'
 'D.l. Hughley' 'Dan Soder' 'Dana Carvey' 'Dane Baptiste' 'Daniel Tosh'
 'Danny Bhoy' 'Dara Ó Briain' 'Darren Harriott' 'Dave Allen' 'Dave Attell'
 'Dave Cha

,full_transcript,artist,show_name,year,source,artist_birthday,artist_gender,age_then
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,1983.0,2,36.0
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,1990.0,2,26.0
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,1970.0,2,43.0
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,1970.0,2,46.0
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,1970.0,2,39.0


In [4]:
### Specific functions to our data - Scraps from the Loft and BBC

# remove notes (Bo Burnham Only)
def clean_bo(text):
    txt = text
    for note in '♫♪':
        txt = txt.replace(note, '')
    return txt

def remove_music(text):
    text = re.sub('♪.*?♪', '', text) # remove ♪ stuff that looks like this ♪
    text = re.sub('♫.*?♫', '', text) # remove ♫ stuff that looks like this ♫
    return text

def remove_bracketed(text):
    text = re.sub('\[.*?\]', '', text) # remove [stuff that looks like this]
    text = re.sub('\(.*?\)', '', text) # remove (stuff that looks like this)
    return text

def remove_speaker_tags(text):
    text = re.sub('\s[\w-]+( \w+)?:\s', ' ', text) # remove Word: or Word word: with a newline or space before
    return text

def remove_info(text):
    text = re.sub('subtitle(s)? by .*', '', str(text)) # remove subtile(s) by xxxx
    text = re.sub('(a)? netflix (original )?(comedy )?(special ?)?', '', text) # remove A Netflix Original Comedy Special
    text = re.sub('(this )?(programme )?(contains )?(very |some )?strong language( |\.)', '', text) # remove strong language
    text = re.sub('adult humou?r( |\.?)?', '', text) # remove adult humour
    text = re.sub('(original )?air date', '', text) # remove air date
    return text

In [5]:
# general functions for text pre-processing
def remove_punc(text, chars):
    txt = text
    for punc in chars:
        txt = txt.replace(punc, '')
    return txt

def remove_num(text):
    return ''.join(char for char in text if not char.isdigit())

def remove_stopw(text, word_list):
    word_tokens = word_tokenize(text)
    return ' '.join(w for w in word_tokens if not w in word_list)

def tokenize(text):
    tokens = word_tokenize(text)
    return tokens

def lemmatize(text):
    lemmatizer = WordNetLemmatizer()
    return ' '.join(lemmatizer.lemmatize(word) for word in text.split(' ') if len(lemmatizer.lemmatize(word))>2)

In [6]:
lemmatizer_dict = {'got': 'get',
                  'gon': 'go',
                  'said': 'say',
                  'fucking': 'fuck',
                  'went': 'go',
                  'finding': 'find'}

def manual_lemmatizer(text):
    for k, v in lemmatizer_dict.items():
        text.replace(k, v)
    return text

In [7]:
clean_df = df.copy()

In [8]:
clean_df

,full_transcript,artist,show_name,year,source,artist_birthday,artist_gender,age_then
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,1983.0,2,36.0
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,1990.0,2,26.0
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,1970.0,2,43.0
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,1970.0,2,46.0
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,1970.0,2,39.0
...,...,...,...,...,...,...,...,...
550,Ladies and gentlemen… !\n\nYes. Yes. Thank you...,Wanda Sykes,Not Normal,2019.0,Scraps from the Loft,1964.0,1,55.0
551,"(techno music playing) ♪ ♪ ♪ Play it, say it, ...",Whitney Cummings,I’m Your Girlfriend,2016.0,Scraps from the Loft,1982.0,1,34.0
552,"Ladies and gentlemen, from the Barclay Theatre...",Whitney Cummings,I Love You,2014.0,Scraps from the Loft,1982.0,1,32.0
553,Ladies and gentlemen… !\n\nThis is awesome. I ...,Whitney Cummings,Can I Touch It?,2019.0,Scraps from the Loft,1982.0,1,37.0


In [9]:
clean_df['full_transcript_clean'] = clean_df['full_transcript'].apply(remove_bracketed)

In [10]:
### clean Bo before removing music
clean_df['full_transcript_clean'][clean_df['artist']=='Bo Burnham'] = clean_df['full_transcript_clean'][clean_df['artist']=='Bo Burnham'].apply(clean_bo)

/var/folders/wx/fdznh_y51_1gnn8tvkcxhsyh0000gn/T/ipykernel_31881/3470318112.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  clean_df['full_transcript_clean'][clean_df['artist']=='Bo Burnham'] = clean_df['full_transcript_clean'][clean_df['artist']=='Bo Burnham'].apply(clean_bo)


In [11]:
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_music)

In [12]:
# lowercase all the words
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].str.lower()

In [13]:
# remove speaker tags and info regexes
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_info)
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_speaker_tags)

In [14]:
clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_num)

In [15]:
### additional words to remove from the scripts
words_to_remove = ['thank', 'cheering', 'recorded', 'applause', 'laughter', 'laughing', 'murmuring', 'chatter',
                       'aired', 'filmed', 'ladies', 'gentlemen', 'welcome', 'stage', 'transcript', 'netflix',
                  'apollo', 'like', 'goodnight', 'mutter', 'noo', 'nuh', 'oof', 'maan', 'fuck', 'cause', 'okay', 
                   'hey', 'also', 'someone', 'somebody', 'everybody', 'also', 'part' , 'sometimes', 'maybe', 
                   'three', 'second', 'everything', 'minute', 'name', 'kind', 'point', 'yeah', 'hello', 'one', 
                   'two', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'whine', 'hnn', 'malla', 'letta', 
                   'namoo', 'getta', 'nama', 'mana', 'chk','manoo', 'hadda', 'ama', 'carlin']
                    
                    # 'know'? 'go'? 'fuck'?
    
                    # haven't left 'i'm' etc. as those should be cleaned up
                    # by a mixture of stopwords, punctuation removeal, lemmatizing and minimum length

stopwords_plus = words_to_remove + stopwords.words('english')

clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_stopw, args=(stopwords_plus,))

In [16]:
punc_added = string.punctuation + '“”‘’…♪♫¶'

clean_df['full_transcript_clean'] = clean_df['full_transcript_clean'].apply(remove_punc, args=(punc_added,))

In [17]:
clean_df

,full_transcript,artist,show_name,year,source,artist_birthday,artist_gender,age_then,full_transcript_clean
0,[rock music playing]\n\n[indistinct chatter]\n...,Adam Devine,Best Time Of Our Lives,2019.0,Scraps from the Loft,1983.0,2,36.0,man let right guys much take ...
1,"strong language. CHEERING Yes, yes, yes! How...",Adam Hess,Live from the BBC,2016.0,BBC,1990.0,2,26.0,yes yes yes well lovely m going apolog...
2,"Ladies and gentlemen, please welcome your hos...",Adam Hills,Live at the Apollo Series 9 Episode 4,2013.0,BBC,1970.0,2,43.0,please host tonight london london live ...
3,some strong language and adult humour Ladies...,Adam Hills,Live at the Apollo Series 12 Episode 4,2016.0,BBC,1970.0,2,46.0,please host tonight nt know hair ok lo...
4,"? CHEERING Hello, Apollo. I am going to start ...",Adam Hills,Live at the Apollo Series 5 Episode 5,2009.0,BBC,1970.0,2,39.0,going start saying something ve probably ne...
...,...,...,...,...,...,...,...,...,...
550,Ladies and gentlemen… !\n\nYes. Yes. Thank you...,Wanda Sykes,Not Normal,2019.0,Scraps from the Loft,1964.0,1,55.0,gentlemen yes yes new york new york oh ...
551,"(techno music playing) ♪ ♪ ♪ Play it, say it, ...",Whitney Cummings,I’m Your Girlfriend,2016.0,Scraps from the Loft,1982.0,1,34.0,please pee good bitches come much sh...
552,"Ladies and gentlemen, from the Barclay Theatre...",Whitney Cummings,I Love You,2014.0,Scraps from the Loft,1982.0,1,32.0,barclay theatre irvine california please ...
553,Ladies and gentlemen… !\n\nThis is awesome. I ...,Whitney Cummings,Can I Touch It?,2019.0,Scraps from the Loft,1982.0,1,37.0,gentlemen awesome shooting fourth standup sp...


In [19]:
clean_df.to_json('../raw_data/all_data_df_stripped.json')

In [18]:
# Convert to list
data = df.full_transcript.values.tolist()

# Remove Emails
data = [re.sub('\S*@\S*\s?', '', sent) for sent in data]

# Remove new line characters
data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

['[rock music playing] [indistinct chatter] Hey, man. How are you? [crowd '
 'cheering] Thank you. Let’s do this. [man] Ladies and gentlemen, ! [loud '
 'cheering] Hey, how’s everybody doing? Thank you, thank you! Thank you! All '
 'right. Yeah! Thank you guys so much. Take a seat. Thank you, thank you. Oh, '
 'my, you guys got me jacked. [crowd whooping] I’m fully juiced for this '
 'thing. Man, I am, I’m so excited to be here. Thank you guys for having me. '
 'I’m gonna have a blast tonight. We all are. [crowd cheers] I’m so excited to '
 'be… I can’t get that excited, though, ’cause I’m an adult man now. '
 '[laughter] You know, you can’t get that excited. They will lock you up. '
 '[laughter] You can’t get little-kid-level, like, geeked, you know? '
 '[laughter] Remember when you were a kid and you would, like, open up a '
 'present, and you didn’t have the vocabulary to say, like, “Thank you so '
 'much, Mama.” [laughter] Or however you guys talked, I don’t know. I was kind '
 'of